## 1. Setup and Imports

In [1]:
import os
import zipfile
import shutil
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.9.0+cu126
CUDA available: False


In [2]:
# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# Device configuration
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

Using device: cpu


## 2. Dataset Handling (Local .zip or Direct Paths)

In [10]:
def extract_zip_dataset(zip_path, extract_to="./dataset"):
    """
    Extract dataset from zip file.
    
    Args:
        zip_path: Path to the .zip file
        extract_to: Directory to extract to
    
    Returns:
        Path to extracted dataset directory
    """
    extract_path = Path(extract_to)
    extract_path.mkdir(parents=True, exist_ok=True)
    
    try:
        print(f"Extracting {zip_path}...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"✓ Dataset extracted successfully")
        return extract_path
    except FileNotFoundError:
        print(f"✗ Zip file not found: {zip_path}")
        raise
    except Exception as e:
        print(f"✗ Error extracting zip file: {str(e)}")
        raise

def find_csv_and_images(dataset_dir):
    """
    Find CSV files and image directory in extracted dataset.
    
    Args:
        dataset_dir: Root directory of dataset
    
    Returns:
        Dictionary with paths to CSVs and image directory
    """
    dataset_path = Path(dataset_dir)
    
    # Find CSV files
    csv_files = list(dataset_path.rglob('*.csv'))
    train_csv = None
    test_csv = None
    
    for csv_file in csv_files:
        if 'train' in csv_file.name.lower():
            train_csv = csv_file
        elif 'test' in csv_file.name.lower():
            test_csv = csv_file
    
    # Find image directories
    image_dirs = []
    for item in dataset_path.rglob('*'):
        if item.is_dir() and any(item.glob('*.jpg')) or any(item.glob('*.png')):
            image_dirs.append(item)
    
    # Use the most common image directory
    image_dir = image_dirs[0] if image_dirs else dataset_path
    
    return {
        'train_csv': train_csv,
        'test_csv': test_csv,
        'image_dir': image_dir
    }

print("Dataset handling functions defined.")

Dataset handling functions defined.


In [ ]:
# ============================================================================
# CONFIGURATION - CHOOSE ONE OPTION BELOW
# ============================================================================

# OPTION 1: Images in ZIP, CSVs in current directory (RECOMMENDED FOR YOUR SETUP)
USE_HYBRID_MODE = True
ZIP_FILE_PATH = './Stall_num_images.zip'  # Images are here (change if different)
EXTRACT_DIR = './images'                   # Extract images to this folder

# OPTION 2: Auto-detect local CSV files and images in same directory
USE_AUTO_DETECT = False

# OPTION 3: Using .zip file (everything in zip)
USE_ZIP = False
EXTRACT_DIR_ZIP = './dataset'

# OPTION 4: Using local paths directly
# USE_AUTO_DETECT = False
# USE_ZIP = False
# USE_HYBRID_MODE = False
# IMAGE_DIR = './Stall_num_images'
# TRAIN_CSV_PATH = './Train.csv'
# TEST_CSV_PATH = './Test.csv'

# ============================================================================
# Common Configuration
# ============================================================================
CONFIG = {
    'batch_size': 4,
    'num_workers': 0,
    'epochs': 100,
    'learning_rate': 0.0001,
    'weight_decay': 0.0001,
    'num_classes': 61,
    'image_size': 224,
    'bbox_loss_weight': 0.01
}

# ------------------------------------------------------------------
# Explicit user paths (override auto-detection)
# Set absolute paths for your CSVs so the notebook finds them reliably.
# Update these if your files are elsewhere.
# ------------------------------------------------------------------
TRAIN_CSV_PATH = '/Users/jainam/Downloads/Neural Network/bject Detection Model using PyTorch/train.csv'
TEST_CSV_PATH = '/Users/jainam/Downloads/Neural Network/bject Detection Model using PyTorch/test.csv'
IMAGE_DIR = None

# Auto-detect a ZIP file for hybrid mode if ZIP_FILE_PATH doesn't exist
cwd = os.getcwd()
if USE_HYBRID_MODE and (not ZIP_FILE_PATH or not os.path.exists(ZIP_FILE_PATH)):
    zip_candidates = [f for f in os.listdir(cwd) if f.lower().endswith('.zip')]
    if zip_candidates:
        ZIP_FILE_PATH = os.path.join(cwd, zip_candidates[0])
        print(f"Auto-detected ZIP file for hybrid mode: {ZIP_FILE_PATH}")

print("Configuration loaded.")
print(f"Mode flags - USE_HYBRID_MODE={USE_HYBRID_MODE}, USE_AUTO_DETECT={USE_AUTO_DETECT}, USE_ZIP={USE_ZIP}")
print(f"TRAIN_CSV_PATH={TRAIN_CSV_PATH}")
print(f"TEST_CSV_PATH={TEST_CSV_PATH}")
print(f"ZIP_FILE_PATH={ZIP_FILE_PATH}")
print(f"EXTRACT_DIR={EXTRACT_DIR}")

SyntaxError: invalid syntax (ipython-input-781616363.py, line 88)

In [12]:
# ============================================================================
# HYBRID MODE: Images from ZIP, CSVs from current directory
# ============================================================================
if USE_HYBRID_MODE:
    print("\n🔄 Hybrid Mode: Extracting images from ZIP, finding CSVs in current directory...\n")
    current_dir = os.getcwd()
    
    # Step 1: Find CSV files in current directory
    print("📄 Step 1: Looking for CSV files in current directory...")
    train_csv_candidates = [f for f in os.listdir(current_dir) if 'train' in f.lower() and f.endswith('.csv')]
    test_csv_candidates = [f for f in os.listdir(current_dir) if 'test' in f.lower() and f.endswith('.csv')]
    
    if train_csv_candidates and test_csv_candidates:
        TRAIN_CSV_PATH = os.path.join(current_dir, train_csv_candidates[0])
        TEST_CSV_PATH = os.path.join(current_dir, test_csv_candidates[0])
        print(f"  ✓ Found Training CSV: {train_csv_candidates[0]}")
        print(f"  ✓ Found Test CSV: {test_csv_candidates[0]}")
    else:
        print(f"  ✗ CSV files not found in current directory")
        print(f"    Found training CSVs: {train_csv_candidates}")
        print(f"    Found test CSVs: {test_csv_candidates}")
    
    # Step 2: Extract images from ZIP file
    print(f"\n📦 Step 2: Extracting images from ZIP file...")
    if os.path.exists(ZIP_FILE_PATH):
        try:
            dataset_dir = extract_zip_dataset(ZIP_FILE_PATH, EXTRACT_DIR)
            IMAGE_DIR = str(dataset_dir)
            print(f"  ✓ Images extracted to: {IMAGE_DIR}")
        except Exception as e:
            print(f"  ✗ Error extracting ZIP: {e}")
            IMAGE_DIR = None
    else:
        print(f"  ✗ ZIP file not found: {ZIP_FILE_PATH}")
        print(f"\nAvailable files in current directory ({current_dir}):")
        for item in os.listdir(current_dir)[:20]:
            print(f"  - {item}")

# ============================================================================
# AUTO-DETECT MODE: Find both CSVs and images in current directory
# ============================================================================
elif USE_AUTO_DETECT:
    print("\n🔍 Auto-detect Mode: Searching for CSV files and images...\n")
    current_dir = os.getcwd()
    print(f"Current directory: {current_dir}\n")
    
    train_csv_candidates = [f for f in os.listdir(current_dir) if 'train' in f.lower() and f.endswith('.csv')]
    test_csv_candidates = [f for f in os.listdir(current_dir) if 'test' in f.lower() and f.endswith('.csv')]
    
    if train_csv_candidates and test_csv_candidates:
        TRAIN_CSV_PATH = os.path.join(current_dir, train_csv_candidates[0])
        TEST_CSV_PATH = os.path.join(current_dir, test_csv_candidates[0])
        
        # Look for image directory
        IMAGE_DIR = None
        for item in os.listdir(current_dir):
            item_path = os.path.join(current_dir, item)
            if os.path.isdir(item_path) and item.lower() not in ['__pycache__', '.config', 'sample_data', '.git']:
                try:
                    if any(f.lower().endswith(('.jpg', '.png', '.jpeg')) for f in os.listdir(item_path)):
                        IMAGE_DIR = os.path.abspath(item_path)
                        break
                except:
                    pass
        
        # If no image dir found, assume images are in current directory
        if IMAGE_DIR is None:
            IMAGE_DIR = current_dir
        
        print(f"✓ Auto-detected paths:")
        print(f"  Train CSV: {TRAIN_CSV_PATH}")
        print(f"  Test CSV: {TEST_CSV_PATH}")
        print(f"  Image directory: {IMAGE_DIR}")
    else:
        print(f"✗ Could not auto-detect CSVs")
        print(f"  Found training CSVs: {train_csv_candidates}")
        print(f"  Found test CSVs: {test_csv_candidates}")
        print(f"\n📝 Please set USE_HYBRID_MODE=True or manually configure paths")

# ============================================================================
# ZIP MODE: Everything in ZIP file
# ============================================================================
elif USE_ZIP:
    print(f"\n📦 ZIP Mode: Attempting to extract ZIP file...\n")
    if not os.path.exists(ZIP_FILE_PATH):
        print(f"✗ Error: {ZIP_FILE_PATH} not found!")
        print(f"\nCurrent working directory: {os.getcwd()}")
        print(f"\nFiles in current directory:")
        for item in os.listdir(os.getcwd())[:20]:  # Show first 20 items
            print(f"  - {item}")
        print(f"\n⚠️  Please upload the ZIP file or set USE_HYBRID_MODE=True")
    else:
        dataset_dir = extract_zip_dataset(ZIP_FILE_PATH, EXTRACT_DIR_ZIP)
        paths_info = find_csv_and_images(dataset_dir)
        
        IMAGE_DIR = str(paths_info['image_dir'])
        TRAIN_CSV_PATH = str(paths_info['train_csv'])
        TEST_CSV_PATH = str(paths_info['test_csv'])
        
        print(f"✓ Dataset paths found:")
        print(f"  Image directory: {IMAGE_DIR}")
        print(f"  Train CSV: {TRAIN_CSV_PATH}")
        print(f"  Test CSV: {TEST_CSV_PATH}")

# ============================================================================
# VERIFY PATHS
# ============================================================================
print(f"\n✓ Verifying paths...")
if TRAIN_CSV_PATH and TEST_CSV_PATH and IMAGE_DIR:
    train_exists = os.path.exists(TRAIN_CSV_PATH)
    test_exists = os.path.exists(TEST_CSV_PATH)
    image_exists = os.path.exists(IMAGE_DIR)
    
    print(f"  Train CSV exists: {train_exists}")
    print(f"  Test CSV exists: {test_exists}")
    print(f"  Image directory exists: {image_exists}")
    
    if train_exists and test_exists and image_exists:
        print(f"\n✓ All paths verified! Ready to load data.")
    else:
        print(f"\n⚠️  Some paths are missing. Please check configuration.")
else:
    print(f"  ❌ Paths not configured properly")
    print(f"  TRAIN_CSV_PATH: {TRAIN_CSV_PATH}")
    print(f"  TEST_CSV_PATH: {TEST_CSV_PATH}")
    print(f"  IMAGE_DIR: {IMAGE_DIR}")


🔄 Hybrid Mode: Extracting images from ZIP, finding CSVs in current directory...

📄 Step 1: Looking for CSV files in current directory...
  ✗ CSV files not found in current directory
    Found training CSVs: []
    Found test CSVs: []

📦 Step 2: Extracting images from ZIP file...
  ✗ ZIP file not found: ./Stall_num_images.zip

Available files in current directory (/content):
  - .config
  - sample_data

✓ Verifying paths...
  ❌ Paths not configured properly
  TRAIN_CSV_PATH: None
  TEST_CSV_PATH: None
  IMAGE_DIR: None


## 3. Data Preprocessing and Augmentation

In [6]:
# Define image transforms
# ImageNet normalization statistics
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
IMAGE_SIZE = CONFIG['image_size']

# Training transforms
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# Test transforms (no augmentation for evaluation)
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

print(f"Transforms configured for image size: {IMAGE_SIZE}x{IMAGE_SIZE}")

Transforms configured for image size: 224x224


## 4. Model Architecture

In [7]:
class ObjectDetectionModel(nn.Module):
    """
    Multi-task learning model for object detection.
    
    Architecture:
    - Backbone: ResNet50 (pretrained on ImageNet)
    - Classification Head: Predicts cow stall numbers (61 classes)
    - Regression Head: Predicts bounding box coordinates (4 values)
    """
    
    def __init__(self, num_classes=61, num_bbox_coords=4):
        super(ObjectDetectionModel, self).__init__()
        
        # Load pretrained ResNet50
        self.backbone = models.resnet50(pretrained=True)
        
        # Get the number of input features for the FC layer
        num_features = self.backbone.fc.in_features
        
        # Replace FC layer with identity to get feature maps
        self.backbone.fc = nn.Identity()
        
        # Flatten layer
        self.flatten = nn.Flatten()
        
        # Classification head
        self.classifier = nn.Linear(num_features, num_classes)
        
        # Bounding box regression head
        self.bbox_regressor = nn.Linear(num_features, num_bbox_coords)
    
    def forward(self, x):
        # Extract features from backbone
        features = self.backbone(x)
        
        # Flatten features
        features = self.flatten(features)
        
        # Classification and bounding box predictions
        class_logits = self.classifier(features)
        bbox_coords = self.bbox_regressor(features)
        
        return class_logits, bbox_coords

print("Model architecture defined.")

Model architecture defined.


## 5. Custom Dataset Class

In [8]:
class CowStallDataset(Dataset):
    """
    Custom dataset for cow stall detection.
    
    Loads images and their corresponding:
    - Stall number labels
    - Bounding box coordinates
    """
    
    def __init__(self, dataframe, image_dir, transform=None):
        """
        Args:
            dataframe: Pandas DataFrame with image paths and annotations
            image_dir: Directory containing images
            transform: Torchvision transforms to apply
        """
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        try:
            # Read image
            image_filename = self.dataframe.iloc[idx, 0]
            image_path = os.path.join(self.image_dir, image_filename)
            
            image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
            if image is None:
                raise ValueError(f"Failed to load image: {image_path}")
            
            # Resize image
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            
            # Convert BGR to RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
            
            # Normalize to [0, 1]
            image /= 255.0
            
            # Apply transforms
            if self.transform:
                image = self.transform(image)
            
            # Get label (stall number)
            label = torch.tensor(int(self.dataframe.iloc[idx, 5]), dtype=torch.long)
            
            # Get bounding box coordinates
            bbox_columns = ['box_position_1', 'box_position_2', 'box_position_3', 'box_position_4']
            bbox = self.dataframe.loc[idx, bbox_columns].values.astype(np.float32)
            
            # Convert (x, y, width, height) to (x1, y1, x2, y2)
            bbox[2] = bbox[0] + bbox[2]  # x2 = x1 + width
            bbox[3] = bbox[1] + bbox[3]  # y2 = y1 + height
            
            bbox = torch.as_tensor(bbox, dtype=torch.float32)
            
            return image, label, bbox
        
        except Exception as e:
            print(f"Error loading sample {idx}: {str(e)}")
            raise

print("Custom dataset class defined.")

Custom dataset class defined.


## 6. Load Data

In [9]:
# Load data (safe checks + helpful diagnostics)
print('
print(f'  TRAIN_CSV_PATH = {TRAIN_CSV_PATH}')
print(f'  TEST_CSV_PATH  = {TEST_CSV_PATH}')
print(f'  ZIP_FILE_PATH  = {ZIP_FILE_PATH if 'ZIP_FILE_PATH' in globals() else None}')
print(f'  IMAGE_DIR      = {IMAGE_DIR}')

# Fallback: try common filenames in cwd if CSV paths are not set
cwd = os.getcwd()
if TRAIN_CSV_PATH is None:
    for name in ['Train.csv', 'train.csv']:
        if os.path.exists(os.path.join(cwd, name)):
            TRAIN_CSV_PATH = os.path.join(cwd, name)
            print(f'  -> Auto-set TRAIN_CSV_PATH to: {TRAIN_CSV_PATH}')
            break

if TEST_CSV_PATH is None:
    for name in ['Test.csv', 'test.csv']:
        if os.path.exists(os.path.join(cwd, name)):
            TEST_CSV_PATH = os.path.join(cwd, name)
            print(f'  -> Auto-set TEST_CSV_PATH to: {TEST_CSV_PATH}')









































        print(f'❌ Error loading data: {e}')    except Exception as e:        print(f'❌ Error: CSV file is empty or malformed: {e}')    except pd.errors.EmptyDataError as e:        print(f'❌ Error: Could not load CSV files: {e}')    except FileNotFoundError as e:        print(f'\n  DataFrame columns: {df_train.columns.tolist()}')        print(f'  Test samples: {len(df_test)}')        print(f'  Training samples: {len(df_train)}')        print(f'\n✓ Data loaded successfully:')        df_test = pd.read_csv(TEST_CSV_PATH).fillna(0)        df_train = pd.read_csv(TRAIN_CSV_PATH).fillna(0)    try:    print('
Loading datasets...')else:    print('  - If images are in a ZIP, ensure ZIP_FILE_PATH points to it and re-run the hybrid extraction cell')    print('  - Update the CONFIG cell to set TRAIN_CSV_PATH, TEST_CSV_PATH, or IMAGE_DIR')    print('  - Place Train.csv and Test.csv into the notebook working directory')    print('Please check one of the following:')    print('
❌ ERROR: Paths are still not configured or files missing.')if not (train_exists and test_exists and image_exists):print(f'  Image dir exists: {bool(image_exists)}')print(f'  Test CSV exists:  {bool(test_exists)}')print(f'  Train CSV exists: {bool(train_exists)}')image_exists = IMAGE_DIR and os.path.exists(IMAGE_DIR)test_exists = TEST_CSV_PATH and os.path.exists(TEST_CSV_PATH)train_exists = TRAIN_CSV_PATH and os.path.exists(TRAIN_CSV_PATH)print('
--- Verifying file existence ---')# Final verification before attempting to load        print(f'  -> Auto-set IMAGE_DIR to ./images: {IMAGE_DIR}')        IMAGE_DIR = os.path.abspath(os.path.join(cwd, 'images'))    elif os.path.exists(os.path.join(cwd, 'images')):        print(f'  -> Auto-set IMAGE_DIR to EXTRACT_DIR: {IMAGE_DIR}')        IMAGE_DIR = os.path.abspath(EXTRACT_DIR)    if os.path.exists(EXTRACT_DIR):if IMAGE_DIR is None:# If IMAGE_DIR not set but extraction dir exists, use that as fallback            break        print(f"❌ Error loading data: {e}")

SyntaxError: unterminated string literal (detected at line 2) (ipython-input-3493565714.py, line 2)

In [ ]:
# Create datasets
if TRAIN_CSV_PATH and TEST_CSV_PATH and IMAGE_DIR:
    try:
        train_dataset = CowStallDataset(
            df_train,
            IMAGE_DIR,
            transform=train_transform
        )

        test_dataset = CowStallDataset(
            df_test,
            IMAGE_DIR,
            transform=test_transform
        )

        # Create data loaders
        train_loader = DataLoader(
            train_dataset,
            batch_size=CONFIG['batch_size'],
            shuffle=True,
            num_workers=CONFIG['num_workers'],
            pin_memory=True if torch.cuda.is_available() else False
        )

        test_loader = DataLoader(
            test_dataset,
            batch_size=CONFIG['batch_size'],
            shuffle=False,
            num_workers=CONFIG['num_workers'],
            pin_memory=True if torch.cuda.is_available() else False
        )

        print(f"\n✓ DataLoaders created:")
        print(f"  Training batches: {len(train_loader)}")
        print(f"  Test batches: {len(test_loader)}")
    except Exception as e:
        print(f"❌ Error creating datasets: {e}")
else:
    print("❌ Cannot create datasets - paths not configured")

❌ Cannot create datasets - paths not configured


## 7. Model Training Setup

In [ ]:
# Initialize model
model = ObjectDetectionModel(
    num_classes=CONFIG['num_classes'],
    num_bbox_coords=4
).to(device)

print(f"Model moved to {device}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 163MB/s]



Model moved to cpu
Total parameters: 23,641,217
Trainable parameters: 23,641,217


In [ ]:
# Loss functions
criterion_classification = nn.CrossEntropyLoss()
criterion_bbox = nn.MSELoss()

# Optimizer
trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(
    trainable_params,
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=30,
    gamma=0.1
)

print(f"✓ Optimizer: {optimizer.__class__.__name__}")
print(f"✓ Scheduler: StepLR (step_size=30, gamma=0.1)")
print(f"✓ Loss functions: CrossEntropyLoss + MSELoss")

✓ Optimizer: Adam
✓ Scheduler: StepLR (step_size=30, gamma=0.1)
✓ Loss functions: CrossEntropyLoss + MSELoss


## 8. Training Loop

In [ ]:
# Training history
history = {
    'train_loss': [],
    'train_accuracy': [],
    'val_loss': [],
    'val_accuracy': []
}

# Hyperparameters
BBOX_LOSS_WEIGHT = CONFIG['bbox_loss_weight']
EPOCHS = CONFIG['epochs']

print(f"Starting training for {EPOCHS} epochs...\n")
print(f"{'Epoch':<8} {'Train Loss':<12} {'Train Acc':<12} {'Val Loss':<12} {'Val Acc':<12}")
print("="*60)

Starting training for 100 epochs...

Epoch    Train Loss   Train Acc    Val Loss     Val Acc     


In [ ]:
for epoch in range(EPOCHS):
    # ==================== Training ====================
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch_idx, (images, labels, bboxes) in enumerate(train_loader):
        # Move data to device
        images = images.to(device)
        labels = labels.to(device)
        bboxes = bboxes.to(device)
        
        # Forward pass
        class_logits, bbox_predictions = model(images)
        
        # Calculate losses
        loss_cls = criterion_classification(class_logits, labels)
        loss_bbox = torch.sqrt(criterion_bbox(bbox_predictions, bboxes)) * BBOX_LOSS_WEIGHT
        loss_total = loss_cls + loss_bbox
        
        # Backward pass
        optimizer.zero_grad()
        loss_total.backward()
        optimizer.step()
        
        # Update metrics
        train_loss += loss_total.item()
        predictions = torch.argmax(class_logits, dim=1)
        train_correct += (predictions == labels).sum().item()
        train_total += labels.size(0)
    
    # Calculate training metrics
    train_loss /= len(train_loader)
    train_accuracy = train_correct / train_total
    history['train_loss'].append(train_loss)
    history['train_accuracy'].append(train_accuracy)
    
    # ==================== Validation ====================
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels, bboxes in test_loader:
            # Move data to device
            images = images.to(device)
            labels = labels.to(device)
            bboxes = bboxes.to(device)
            
            # Forward pass
            class_logits, bbox_predictions = model(images)
            
            # Calculate losses
            loss_cls = criterion_classification(class_logits, labels)
            loss_bbox = torch.sqrt(criterion_bbox(bbox_predictions, bboxes)) * BBOX_LOSS_WEIGHT
            loss_total = loss_cls + loss_bbox
            
            # Update metrics
            val_loss += loss_total.item()
            predictions = torch.argmax(class_logits, dim=1)
            val_correct += (predictions == labels).sum().item()
            val_total += labels.size(0)
    
    # Calculate validation metrics
    val_loss /= len(test_loader)
    val_accuracy = val_correct / val_total
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_accuracy)
    
    # Update learning rate
    scheduler.step()
    
    # Print progress every 10 epochs
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"{epoch + 1:<8} {train_loss:<12.4f} {train_accuracy*100:<12.2f} {val_loss:<12.4f} {val_accuracy*100:<12.2f}")

NameError: name 'train_loader' is not defined

In [ ]:
print("\n" + "="*80)
print("✓ TRAINING COMPLETED")
print("="*80)
print(f"Final Training Loss: {history['train_loss'][-1]:.4f}")
print(f"Final Training Accuracy: {history['train_accuracy'][-1]*100:.2f}%")
print(f"Final Validation Loss: {history['val_loss'][-1]:.4f}")
print(f"Final Validation Accuracy: {history['val_accuracy'][-1]*100:.2f}%")

## 9. Model Evaluation and Visualization

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Loss plot
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(history['train_accuracy'], label='Train Accuracy', linewidth=2)
axes[1].plot(history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Find best validation accuracy
best_epoch = np.argmax(history['val_accuracy']) + 1
best_val_acc = np.max(history['val_accuracy'])

print(f"\nBest Validation Accuracy: {best_val_acc*100:.2f}% (Epoch {best_epoch})")
if best_val_acc > 0.8:
    print(f"✓ Model exceeds 80% accuracy requirement")
else:
    print(f"✗ Below 80% accuracy")

## 10. Model Saving

In [ ]:
# Create models directory if it doesn't exist
os.makedirs('./models', exist_ok=True)

# Save model checkpoint
model_save_path = './models/object_detection_model.pt'

checkpoint = {
    'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'train_loss': history['train_loss'][-1],
    'val_loss': history['val_loss'][-1],
    'val_accuracy': history['val_accuracy'][-1],
    'config': CONFIG,
    'history': history
}

try:
    torch.save(checkpoint, model_save_path)
    print(f"✓ Model saved successfully to: {model_save_path}")
except Exception as e:
    print(f"✗ Error saving model: {str(e)}")

In [ ]:
# Load model function for future use
def load_model(checkpoint_path, device):
    """
    Load model from checkpoint.
    
    Args:
        checkpoint_path: Path to saved checkpoint
        device: Device to load model on
    
    Returns:
        Loaded model and checkpoint information
    """
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    model = ObjectDetectionModel(
        num_classes=checkpoint['config']['num_classes'],
        num_bbox_coords=4
    ).to(device)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    return model, checkpoint

print("Model loading function defined.")

## 11. Summary and Results

In [ ]:
print("\n" + "="*80)
print("PROJECT SUMMARY")
print("="*80)
print(f"\n📊 Dataset: Cow Stall Number Detection")
print(f"   Training samples: {len(df_train)}")
print(f"   Test samples: {len(df_test)}")
print(f"\n🔧 Model Architecture: ObjectDetectionModel")
print(f"   Backbone: ResNet50 (pretrained)")
print(f"   Classification classes: {CONFIG['num_classes']}")
print(f"   Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"\n⚙️  Training Configuration:")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {CONFIG['batch_size']}")
print(f"   Learning rate: {CONFIG['learning_rate']}")
print(f"   Optimizer: Adam")
print(f"\n📈 Final Results:")
print(f"   Training Loss: {history['train_loss'][-1]:.4f}")
print(f"   Training Accuracy: {history['train_accuracy'][-1]*100:.2f}%")
print(f"   Validation Loss: {history['val_loss'][-1]:.4f}")
print(f"   Validation Accuracy: {history['val_accuracy'][-1]*100:.2f}%")
print(f"   Best Validation Accuracy: {best_val_acc*100:.2f}% (Epoch {best_epoch})")
print(f"\n{'✓' if best_val_acc > 0.8 else '✗'} {'Model meets' if best_val_acc > 0.8 else 'Below'} 80% accuracy requirement")
print(f"\n💾 Model saved to: {model_save_path}")
print("="*80)